# Template Jupyter Notebook

In [436]:
import json
import codecs
import unicodedata
import re
import pandas as pd

In [437]:
def parse_json(path: str):
    with open(path, "r", encoding="utf-8") as file:
        raw_data = json.load(file)
    
    messages = pd.DataFrame(raw_data["messages"])
    participants = pd.DataFrame(raw_data["participants"])
    
    messages["content"] = messages["content"].apply(clean_text)
    messages["sender_name"] = messages["sender_name"].apply(clean_text)
    participants["name"] = participants["name"].apply(clean_text)
    
    return messages, participants 

In [438]:
def get_conversation(messages: pd.DataFrame, totalConversation: dict):
    for index, row in messages.iterrows():
        sender = row.get("sender_name")
        content = row.get("content")
        if content == None:
            continue
        totalConversation.setdefault(sender, []).append(content)
    return totalConversation

In [439]:
def clean_text(text):
    if not isinstance(text, str):
        return text
    try:
        text = text.encode("latin1").decode("utf-8")
    except Exception:
        pass
    text = text.replace("\u200d", "")   # remove zero-width joiner
    return text

In [440]:
def word_frequency(conversations: dict, targetWord: str):
    targetWord = targetWord.lower().strip()
    word_count = {}

    for sender, messages in conversations.items():  # ← FIXED
        count = 0
        for msg in messages:
            # Skip invalid values
            if not isinstance(msg, str) or not msg.strip():
                continue

            lower_msg = msg.lower()

            # Count if target word appears
            if targetWord in lower_msg:
                count += 1

        # Always include user even if count = 0
        word_count[sender] = count

    return word_count

In [441]:
messages, participants = parse_json("message_1.json")
result = get_conversation(messages, {})
word_freq = word_frequency(result, "fuck")
print(word_freq)

{'Austen Brown': 2, '𝙹𝚘𝚛𝚍𝚘𝚗': 58, 'alex.edwardss': 39, 'Phoenix': 1, 'Bryce Love': 3, 'abil': 1, 'Jonathan Sheehy': 5, 'Aidan': 0, 'Krop': 2, 'clayton': 0, 'Ali': 0, 'shayna': 8, 'Noman Beckett': 7, 'Zuniel Sali™': 0, 'Myles': 0, 'T A Y L O R': 0, 'Elijah': 7, 'captain_jackpotatoes': 0, 'Kaitlyn👑': 0, 'Maddy 🧟♀️': 0, 'Will Shaw': 1, 'Hayden McCormick': 0, 'Instagram User': 0, 'Ewan': 0, 'Meta AI': 0, 'eeugenio.maldonado': 1, 'Kyle Steabner': 0, 'Hayden Lindsay': 0, 'ali': 0, 'Torin': 0}
